In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.12:0.14.0 pyspark-shell'

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
df = spark.read.format('xml').options(rowTag='page').load('hdfs:/enwiki_test.xml')

:: loading settings :: url = jar:file:/usr/lib/spark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.databricks#spark-xml_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-dcaff25a-96a7-4d3e-9483-b52550ed726b;1.0
	confs: [default]
	found com.databricks#spark-xml_2.12;0.14.0 in central
	found commons-io#commons-io;2.8.0 in central
	found org.glassfish.jaxb#txw2;2.3.4 in central
	found org.apache.ws.xmlschema#xmlschema-core;2.2.5 in central
:: resolution report :: resolve 290ms :: artifacts dl 8ms
	:: modules in use:
	com.databricks#spark-xml_2.12;0.14.0 from central in [default]
	commons-io#commons-io;2.8.0 from central in [default]
	org.apache.ws.xmlschema#xmlschema-core;2.2.5 from central in [default]
	org.glassfish.jaxb#txw2;2.3.4 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|

In [3]:
df.show()

+-------+--------+------------+--------------------+--------------------+
|     id|redirect|restrictions|            revision|               title|
+-------+--------+------------+--------------------+--------------------+
|1600001|    null|        null|{{Robot - Removin...|Wikipedia:Article...|
|1600006|    null|        null|{{[[WP:CHECKWIKI]...|      Homer Jacobson|
|1600009|        |        null|{{{{R from other ...|             Deflate|
|1600011|        |        null|{{[[WP:AES|←]]Red...|Hanna-Barbera Car...|
|1600017|        |        null|{null, {null, nul...|          Facel-Vega|
|1600021|        |        null|{null, {null, nul...|Birmingham Railwa...|
|1600022|    null|        null|{{[[WP:CHECKWIKI]...|     Edward F. Moore|
|1600027|    null|        null|{null, {null, nul...|      Tyler Florence|
|1600028|        |        null|{{{{R from other ...|Accentual-Syllabi...|
|1600031|        |        null|{{[[WP:AES|←]]Red...|          Blackwhite|
|1600034|    null|        null|{{Inter

In [4]:
df.printSchema()

root
 |-- id: long (nullable = true)
 |-- redirect: string (nullable = true)
 |-- restrictions: string (nullable = true)
 |-- revision: struct (nullable = true)
 |    |-- comment: struct (nullable = true)
 |    |    |-- _VALUE: string (nullable = true)
 |    |    |-- _deleted: string (nullable = true)
 |    |-- contributor: struct (nullable = true)
 |    |    |-- _VALUE: string (nullable = true)
 |    |    |-- _deleted: string (nullable = true)
 |    |    |-- id: long (nullable = true)
 |    |    |-- ip: string (nullable = true)
 |    |    |-- username: string (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- minor: string (nullable = true)
 |    |-- text: struct (nullable = true)
 |    |    |-- _VALUE: string (nullable = true)
 |    |    |-- _xml:space: string (nullable = true)
 |    |-- timestamp: timestamp (nullable = true)
 |-- title: string (nullable = true)



In [17]:
import re
from pyspark.sql.functions import udf, col, explode

def extractLink(text):
    try:
        results = re.findall(r'(?<=\[\[).*?(?=\]\])', text)
    except:
        results = []
    output = []
    for res in results:
        if ':' in res and 'Category:' not in res:
            continue
        else:
            temp = res.split('|') #all links
            valid_links = [ x.lower() for x in temp if "#" not in x]
            if len(valid_links)==0:
                continue
            else:
                output.append(valid_links[0])
    return output

In [18]:
from pyspark.sql.types import StringType, ArrayType

link_udf = udf(lambda text: extractLink(text), ArrayType(StringType()))
newdf = df.withColumn("article", explode(link_udf(col("revision.text._VALUE"))))

In [19]:
newdf.printSchema()

root
 |-- id: long (nullable = true)
 |-- redirect: string (nullable = true)
 |-- restrictions: string (nullable = true)
 |-- revision: struct (nullable = true)
 |    |-- comment: struct (nullable = true)
 |    |    |-- _VALUE: string (nullable = true)
 |    |    |-- _deleted: string (nullable = true)
 |    |-- contributor: struct (nullable = true)
 |    |    |-- _VALUE: string (nullable = true)
 |    |    |-- _deleted: string (nullable = true)
 |    |    |-- id: long (nullable = true)
 |    |    |-- ip: string (nullable = true)
 |    |    |-- username: string (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- minor: string (nullable = true)
 |    |-- text: struct (nullable = true)
 |    |    |-- _VALUE: string (nullable = true)
 |    |    |-- _xml:space: string (nullable = true)
 |    |-- timestamp: timestamp (nullable = true)
 |-- title: string (nullable = true)
 |-- article: string (nullable = true)



In [21]:
newdf.select('title', 'article').repartition(10).write.option("delimiter", "\t").csv('p2t2_test')